In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tomotok.core.derivative import compute_iso_dmats, compute_aniso_dmats
from tomotok.core.phantoms import gauss_iso, iso_psi, gauss
from tomotok.core.geometry import sparse_line_3d, RegularGrid, generate_los
from tomotok.core.inversions import CholmodMfr, Mfr

In [ ]:
grid = RegularGrid(25, 50, (.2, .7), (-.5, .5))

In [ ]:
# phantom = gauss_iso(grid.nx, grid.ny, cen=.3, amp=100)
ipsi = iso_psi(grid.nr, grid.nz, span=1.2)  # circular "flux" surfaces
phantom = gauss(ipsi, cen=.3, amp=100)  # should be the same as gauss_iso

In [ ]:
plt.figure()
plt.imshow(phantom, extent=grid.extent, origin='lower')
plt.colorbar()
plt.show()

In [ ]:
num = 20
ss = []
es = []
s1, e1 = generate_los(num=(num, 1), fov=(70, 0), pinhole=(1, 0, 0), axis=(-1, 0, 0))
s2, e2 = generate_los(num=(num, 1), fov=(35, 0), pinhole=(.4, 0, .8), axis=(.1, 0, -1), elong=1.5)
s3, e3 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.75, 0, -.4), axis=(-1, 0, 1), elong=1.5)
s4, e4 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.75, 0, .4), axis=(-1, 0, -1), elong=1.5)
ss.append(s1)
es.append(e1)
ss.append(s2)
es.append(e2)
ss.append(s3)
es.append(e3)
es.append(e4)
ss.append(s4)
s = np.concatenate(ss, 0)
e = np.concatenate(es, 0)

In [ ]:
rch = np.empty((s.shape[0], 2))
zch = np.empty((s.shape[0], 2))
rch[:, 0] = s[:, 0]
rch[:, 1] = e[:, 0]
zch[:, 0] = s[:, 2]
zch[:, 1] = e[:, 2]

In [ ]:
gmat = sparse_line_3d(rch, zch, grid, rmin=.2)

In [ ]:
plt.figure()
plt.imshow(gmat.toarray().sum(0).reshape(grid.shape), extent=grid.extent, origin='lower')
plt.colorbar()
plt.show()

In [ ]:
sig = gmat.dot(phantom.flatten())

In [ ]:
plt.plot(sig)
plt.show()

In [ ]:
data = sig.reshape(1, -1)  # data should have shape (#timeslices, #channels/pixels)

In [ ]:
try: 
    solver = CholmodMfr()  # requires scikit.sparse
except ImportError:
    solver = Mfr()  # slowlier version based on scipy.sparse

In [ ]:
errors = (data + data.max() ) / 2 * .05
# errors = .001

In [ ]:
# dmats = compute_iso_dmats(grid)
dmats = compute_aniso_dmats(grid, magflux=ipsi)

In [ ]:
res, chi = solver(data, gmat, dmats, errors, danis=0)

In [ ]:
f, a = plt.subplots()
# img = a.imshow(res.reshape(grid.shape), origin='lower', extent=grid.extent)  # nodes must be squares
img = a.pcolorfast(grid.r_center, grid.z_center, res[0].reshape(grid.shape))
a.set_aspect(1)
a.set_xlabel('R [-]')
a.set_ylabel('z [-]')
f.colorbar(img, ax=a)
plt.show()

In [ ]:
levels = np.linspace(0, res[0].max(), 10)
plt.figure()
plt.contour(res.reshape(grid.shape), levels=levels)
plt.contour(phantom, linestyles='dotted', levels=levels)
plt.show()

In [ ]:
'Emissivity sum MFR: {:.2f} and phantom: {:.2f}'.format(res[0].sum(), phantom.sum())